<a href="https://colab.research.google.com/github/String-ru/AIFFEL_quest_cr/blob/main/Exploration/Quest01/FlutterConnect_Flower_better.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
# 1. 상수 정의
BATCH_SIZE = 32
IMG_SIZE = 224

In [ ]:
# 2. 데이터 로드
(raw_train, raw_val, raw_test), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)
class_names = metadata.features['label'].names

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/tf_flowers/incomplete.22S8F8_3.0.1/tf_flowers-train.tfrecord*...:   0%|   …

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [ ]:
# 3. 전처리 & 증강
def format_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    k = tf.random.uniform([], 0, 4, dtype=tf.int32)
    image = tf.image.rot90(image, k)
    crop_size = tf.random.uniform([], int(0.8*IMG_SIZE), IMG_SIZE, dtype=tf.int32)
    image = tf.image.random_crop(image, [crop_size, crop_size, 3])
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    return image, label

In [ ]:
# 4. 데이터 배치 준비
train_batches = (
    raw_train
    .map(format_image, tf.data.AUTOTUNE)
    .map(augment, tf.data.AUTOTUNE)
    .shuffle(1000)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)
validation_batches = raw_val.map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_batches       = raw_test.map(format_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# 5. 모델 구성 (Fine‑tuning)
base_model = VGG16(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                   include_top=False, weights='imagenet')
base_model.trainable = True
for layer in base_model.layers:
    layer.trainable = layer.name.startswith('block5_')

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=l2(1e-4)),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# 6. 컴파일 & 학습
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_batches,
    epochs=15,
    validation_data=validation_batches
)

Epoch 1/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 57s 405ms/step - accuracy: 0.3295 - loss: 1.9837 - val_accuracy: 0.4986 - val_loss: 1.2495
Epoch 2/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 24s 213ms/step - accuracy: 0.5793 - loss: 1.2144 - val_accuracy: 0.7084 - val_loss: 0.8809
Epoch 3/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 23s 204ms/step - accuracy: 0.6677 - loss: 0.9579 - val_accuracy: 0.7711 - val_loss: 0.7013
Epoch 4/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 215ms/step - accuracy: 0.7022 - loss: 0.8476 - val_accuracy: 0.7847 - val_loss: 0.6390
Epoch 5/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 40s 207ms/step - accuracy: 0.7514 - loss: 0.7196 - val_accuracy: 0.8065 - val_loss: 0.5663
Epoch 6/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 217ms/step - accuracy: 0.7873 - loss: 0.6185 - val_accuracy: 0.8256 - val_loss: 0.5342
Epoch 7/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 22s 212ms/step - accuracy: 0.7842 - loss: 0.6513 - val_accuracy: 0.8392 - val_loss: 0.4907
Epoch 8/15
92/92 ━━━━━━━━━━━━━━━━━━━━ 42s 221ms/step - accuracy: 0.7940 - loss: 0.6042 - val_accu

In [ ]:
# 7. 테스트 세트 성능 평가
test_loss, test_acc = model.evaluate(test_batches)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc*100:.2f}%")

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.8786 - loss: 0.3444
Test Loss: 0.3274
Test Accuracy: 89.37%


In [ ]:
import tensorflow as tf

# 훈련된 모델을 TFLite 형식으로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# TFLite 모델을 파일로 저장
with open('flower_classifier.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp38ctiw9b'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_19')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  136862651047696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651049616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651049424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651049808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651050192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651049232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651049040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651050384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651051344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136862651052112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13686265105

In [ ]:
# 클래스 레이블을 텍스트 파일로 저장 (한 줄에 하나씩)
with open('labels.txt', 'w') as f:
    for name in class_names:
        f.write(name + '\n')

In [ ]:
from google.colab import files

# 파일 다운로드
files.download('flower_classifier.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>